# A functional burst search and download service

## Setup

In [ ]:
import bursts
import asf_search as asf
from datetime import datetime, timedelta
import xarray as xr
from pathlib import Path
import numpy as np
from importlib import reload
import geopandas as gpd
from shapely import geometry, wkt
import time
import pystac
import fsspec
import xml.etree.ElementTree as ET

In [ ]:
safe_url = 'https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_SLC__1SDV_20220716T115019_20220716T115049_044125_054459_71A9.zip'

## Demoing interior objects
These objects will not likely not be accessed by users, but this function demonstrates how the metadata objects are created

In [ ]:
auth = bursts.get_netrc_auth()

In [ ]:
manifest, annotations = bursts.edl_download_metadata(safe_url, auth)

In [ ]:
slc = bursts.SLCMetadata(safe_url, manifest, annotations)

In [ ]:
swath = bursts.SwathMetadata(slc,'vv',swath_index=0)

In [ ]:
burst = bursts.BurstMetadata(swath,burst_index=0)

## Creating burst metadata datasets
Users have the option of creating either a geodatframe, or a STAC catalog. Burst data download is only enabled for the STAC catalog, but the geodataframe is great for in-notebook visualizations. Hopefully, users won't have to create these datasets themselves and will instead use an ASF-provided dataset.

In [ ]:
start = datetime.strptime('20210101','%Y%m%d')
end = datetime.strptime('20210201','%Y%m%d')
mt_edgecumb = 'POLYGON ((-135.7849 57.0417,-135.7243 57.0417,-135.7243 57.0654,-135.7849 57.0654,-135.7849 57.0417))'
insar_opts = dict(platform=[asf.PLATFORM.SENTINEL1], polarization=['VV','VV+VH','Dual VV'], beamMode='IW', processingLevel='SLC')
results = asf.geo_search(flightDirection='DESCENDING', intersectsWith=mt_edgecumb, start=start, end=end, **insar_opts)
safe_urls = [x.properties['url'] for x in results]

In [ ]:
safe_urls

In [ ]:
burst_list = bursts.get_burst_metadata(safe_urls)

In [ ]:
stac = bursts.generate_burst_stac_catalog(burst_list)
stac.describe()

## Explore with Geopandas

In [ ]:
items = [x.to_dict() for x in stac.get_all_items()]
df = gpd.GeoDataFrame.from_features(items, crs="epsg:4326")
df.index = [x['id'] for x in items]

In [ ]:
in_aoi = df.intersects(wkt.loads(mt_edgecumb))
aoi = df.loc[in_aoi].copy()
print(f'Located {aoi.shape[0]} bursts that meet criteria')

In [ ]:
# aoi.explore()

## View using STAC-browser

In [ ]:
stac_path = bursts.save_stac_catalog_locally(stac)

In [ ]:
bursts.initiate_stac_catalog_server(5555,stac_path)

## Download burst data
Using the STAC catalog, users can select an STAC item, then pass this item to `bursts.edl_download_burst_data`. The data is returned in a 2d numpy array with a complex float data type.

In [ ]:
items = [stac.get_item(x,recursive=True) for x in aoi.index]

In [ ]:
import time
start = time.time()
burst_data = [bursts.edl_download_burst_data(x,'VV') for x in items]
end = time.time()
print(end-start)

## xarray-sentinel

In [ ]:
slc_path = Path('../example_data/S1B_IW_SLC__1SDV_20210131T151555_20210131T151621_025400_03067B_415D.SAFE').resolve()
dummy1 = xr.open_dataset(slc_path, group="IW1/VH", engine="sentinel-1")
dummy2 = xr.open_dataset(slc_path, group="IW1/VH", engine="sentinel-1")

In [ ]:
dummy1.attrs['date'] = datetime.strptime(dummy1.attrs['start_time'], '%Y-%m-%dT%H:%M:%S.%f')
dummy2.attrs['date'] = datetime.strptime(dummy2.attrs['start_time'], '%Y-%m-%dT%H:%M:%S.%f')
dummy2.attrs['date'] += timedelta(days=30)

In [ ]:
dummy1.variables

In [ ]:
dummy1.set_coords()

In [ ]:
xr.concat([dummy1,dummy2],dim='time',coords={'time':[dummy1.attrs['date'], dummy2.attrs['date']]})

## xarray to zarr

In [ ]:
n_lines, n_samples = burst_data[0].shape
dates = [x.datetime for x in items]
dims = ('time','line','sample')
coords = (dates,list(range(n_lines)),list(range(n_samples)))
coords = {key:value for key,value in zip(dims,coords)}

In [ ]:
burst_array = xr.Dataset(burst_data,dims=('time','line','sample'),coords=coords)